In [61]:
import causaldata
import numpy as np
print(dir(causaldata))
df = causaldata.yule.load_pandas().data
print(causaldata.yule.NOTE)
print(causaldata.yule.DESCRLONG)
print(causaldata.yule.SOURCE)

['Mroz', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'abortion', 'adult_services', 'auto', 'avocado', 'black_politicians', 'castle', 'close_elections_lmb', 'cps_mixtape', 'credit_cards', 'gapminder', 'google_stock', 'gov_transfers', 'gov_transfers_density', 'greek_data', 'mortgages', 'nhefs', 'nhefs_codebook', 'nhefs_complete', 'nsw_mixtape', 'organ_donations', 'restaurant_inspections', 'ri', 'scorecard', 'snow', 'social_insure', 'texas', 'thornton_hiv', 'titanic', 'training_bias_reduction', 'training_example', 'yule']
::
    Number of observations - 32
    Number of variables - 5
    Variables name definitions::
            location - Location in England
            paup - Pauperism Growth
            outrelief - Poverty Relief Growth
            old - Annual growth in aged population
            pop - Annual growth in population

This data allows for a look at the correlation between poverty relief a

In [62]:
outcome = 'paup'
treatment = 'outrelief'
rest = ['old', 'pop']

In [63]:
#import wooldridge
#df = wooldridge.data('jtrain3')
#df['avg'] = 0.5 * (df.re74+df.re75)
#df = df.dropna()
#df = df[df.avg <= 15]
y = df[outcome]
d = df[treatment]
x = df[rest]
print(df.shape)
df.head()

(32, 5)


,location,paup,outrelief,old,pop
0,Kensington,27,5,104,136
1,Paddington,47,12,115,111
2,Fulham,31,21,85,174
3,Chelsea,64,21,81,124
4,St. George’s,46,18,113,96


In [64]:
# Simple Comparision of Means
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:                   paup   R-squared:                       0.353
Model:                            OLS   Adj. R-squared:                  0.331
Method:                 Least Squares   F-statistic:                     16.36
Date:                Sun, 04 Dec 2022   Prob (F-statistic):           0.000338
Time:                        20:41:24   Log-Likelihood:                -127.62
No. Observations:                  32   AIC:                             259.2
Df Residuals:                      30   BIC:                             262.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.7654      0.189      4.045      0.0

In [65]:
# Pooled Regression Adjustment
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d, x], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:                   paup   R-squared:                       0.697
Model:                            OLS   Adj. R-squared:                  0.665
Method:                 Least Squares   F-statistic:                     21.49
Date:                Sun, 04 Dec 2022   Prob (F-statistic):           2.00e-07
Time:                        20:41:26   Log-Likelihood:                -115.47
No. Observations:                  32   AIC:                             238.9
Df Residuals:                      28   BIC:                             244.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.7521      0.135      5.572      0.0

In [66]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col=outcome,
                                  d_cols=treatment,
                                  x_cols=rest)
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 6)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(3141)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: paup
Treatment variable(s): ['outrelief']
Covariates: ['old', 'pop']
Instrument variable(s): None
No. Observations: 32

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 31
Columns: 5 entries, location to pop
dtypes: int16(2), int8(2), object(1)
memory usage: 704.0+ bytes

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: paup
Treatment variable(s): ['outrelief']
Covariates: ['old', 'pop']
Instrument variable(s): None
No. Observations: 32

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor(max_depth=6, max_features='sqrt', n_estimators=500)
Learner ml_m: R

In [59]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=45)
est.fit(y, d, X=None,W=x)
est.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,0.7,0.102,6.847,0.0,0.5,0.901
